## 1. Setup

In [42]:
import os
import sys
sys.path.append("C:\\Miniconda\\envs\\kogas_env1\\Lib\\site-packages")
import shutil
import datetime
import random as rnd
from glob import glob

import numpy as np
from numpy import random as np_rnd
import pandas as pd
from scipy.stats import linregress
from scipy.stats import trim_mean

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import metrics as skl_merics

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
# import seaborn as sns

import warnings
warnings.simplefilter(action="ignore")

pd.set_option("display.max_rows", 500)
# pd.set_option("display.height", 500)

plt.rcParams["axes.unicode_minus"] = False
font_path = "C:\\Users\\kogas\\Desktop\\nanum-square\\NanumSquareR.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [43]:
def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    rnd.seed(seed)
    np_rnd.seed(seed)

def diff(x1, x2):
    x2 = set(x2)
    return [i for i in x1 if i not in x2]

In [44]:
folder_path = "C:\\Users\\kogas\\Desktop\\jupyter_root_folder\\YJ_notebooks\\"
# C:\Users\kogas\Desktop\jupyter_root_folder\YJ_notebooks
seed_everything()

**data loading**

In [45]:
df_supply = pd.read_csv(folder_path + '\\dataset\\월별공급량및비중.csv')
df_supply.columns = ["year", "month", "target_civil", "target_ind", "total", "weight_civil", "weight_ind"]
df_indust = pd.read_csv(folder_path + '\\dataset\\제조업 부가가치(분기별).csv')
df_indust.columns = ["year", "quarter", "qva"]
df_commer = pd.read_csv(folder_path + '\\dataset\\상업용 상대가격(기준=2015).csv')
df_commer.columns = ["year", "month", "relative_price", "gas_price", "oil_price"]

**시간 관련 feature 입력**

In [46]:
quarter_dic = {
    "Q1": [1, 2, 3],
    "Q2": [4, 5, 6],
    "Q3": [7, 8, 9],
    "Q4": [10, 11, 12],
}
# df_indust 에는 쿼터가 없기 때문에, 다른 데이터에 쿼터를 달아준다
month_dic = {
    1 : 'Q1',
    2 : 'Q1',
    3 : 'Q1',
    4 : 'Q2',
    5 : 'Q2',
    6 : 'Q2',
    7 : 'Q3',
    8 : 'Q3',
    9 : 'Q3',
    10 : 'Q4',
    11 : 'Q4',
    12 : 'Q4'
}

In [47]:
df_supply["quarter"] = df_supply["month"].apply(lambda x: month_dic[x])
df_indust["month"] = df_indust["quarter"].apply(lambda x: quarter_dic[x])
df_commer["quarter"] = df_commer["month"].apply(lambda x: month_dic[x])

In [48]:
df_supply.head()

,year,month,target_civil,target_ind,total,weight_civil,weight_ind,quarter
0,1996,1,605519.0,83809.0,689328.0,0.87842,0.12158,Q1
1,1996,2,566323.0,70427.0,636750.0,0.88940,0.11060,Q1
2,1996,3,477514.0,62652.0,540166.0,0.88401,0.11599,Q1
3,1996,4,337794.0,47050.0,384844.0,0.87774,0.12226,Q2
4,1996,5,184522.0,30709.0,215231.0,0.85732,0.14268,Q2


In [49]:
df_indust.head()

,year,quarter,qva,month
0,1996,Q1,36550.3,"[1, 2, 3]"
1,1996,Q2,37152.4,"[4, 5, 6]"
2,1996,Q3,37792.4,"[7, 8, 9]"
3,1996,Q4,38372.4,"[10, 11, 12]"
4,1997,Q1,38710.8,"[1, 2, 3]"


In [50]:
df_indust = df_indust.explode("month")
df_indust["qva"] /= 3

In [51]:
df_indust.head()

,year,quarter,qva,month
0,1996,Q1,12183.433333,1
0,1996,Q1,12183.433333,2
0,1996,Q1,12183.433333,3
1,1996,Q2,12384.133333,4
1,1996,Q2,12384.133333,5


In [52]:
df_commer.head()

,year,month,relative_price,gas_price,oil_price,quarter
0,1996,1,0.97,26.94,27.86,Q1
1,1996,2,0.93,26.94,29.04,Q1
2,1996,3,0.96,26.94,27.99,Q1
3,1996,4,0.94,26.94,28.74,Q2
4,1996,5,0.92,26.94,29.18,Q2


In [53]:
df_full = df_supply[["year", "quarter", "month"]]
concat_list = [df_supply, df_indust, df_commer]
for i in concat_list:  
    df_full = pd.merge(df_full, i, on=["year", "quarter", "month"], how="left")

In [54]:
# df_full.columns = ["year", "month", "target_civil", "target_ind", "total", "weight_civil", "weight_ind", "quarter", "qva", "qva_norm2015", "relative_price", "price_gas", "price_oil"]
df_full = df_full.drop(["weight_ind", "total"], axis=1)
# df_test.columns = ["year", "month", "target_civil", "target_ind"]

In [55]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            300 non-null    int64  
 1   quarter         300 non-null    object 
 2   month           300 non-null    object 
 3   target_civil    300 non-null    float64
 4   target_ind      300 non-null    float64
 5   weight_civil    300 non-null    float64
 6   qva             300 non-null    float64
 7   relative_price  300 non-null    float64
 8   gas_price       300 non-null    float64
 9   oil_price       300 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 25.8+ KB


In [56]:
df_full.head()

,year,quarter,month,target_civil,target_ind,weight_civil,qva,relative_price,gas_price,oil_price
0,1996,Q1,1,605519.0,83809.0,0.87842,12183.433333,0.97,26.94,27.86
1,1996,Q1,2,566323.0,70427.0,0.88940,12183.433333,0.93,26.94,29.04
2,1996,Q1,3,477514.0,62652.0,0.88401,12183.433333,0.96,26.94,27.99
3,1996,Q2,4,337794.0,47050.0,0.87774,12384.133333,0.94,26.94,28.74
4,1996,Q2,5,184522.0,30709.0,0.85732,12384.133333,0.92,26.94,29.18


## 외부데이터 로딩

In [ ]:
external_data = {
    "tmper": None,
    "hum": None,
    "eura_snow": None,
    "neg_north": None,
    "sea_ice": None,
}

**온도**

In [ ]:
tmp_df = []

for i in glob("C:\\Users\\kogas\\Desktop\\external_data\\20220930\\기온\\*"):
    if i.split("\\")[-1][:2] in ["서울", "인천", "대전", "대구", "광주", "울산", "부산"]:
        tmp = pd.read_csv(i, encoding="cp949")
        tmp = tmp.iloc[:, 1:]
        tmp.columns = ["지점명", "일시", "평균기온", "최고기온", "최고기온시각", "최저기온", "최저기온시각", "일교차"]
        tmp = tmp[["지점명", "일시", "평균기온", "최고기온", "최저기온", "일교차"]]
        tmp["일시"] = pd.to_datetime(tmp["일시"])
        tmp["year"] = tmp["일시"].dt.year
        tmp["month"] = tmp["일시"].dt.month
        tmp = tmp.loc[(tmp["일시"] >= pd.to_datetime("1996-01-01")) &( tmp["일시"] <= pd.to_datetime("2020-12-31"))]
        tmp_df.append(tmp)

tmp_df = pd.concat(tmp_df, axis=0, ignore_index=True)

In [ ]:
df_full[["전국평균_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.groupby(["year", "month"]).mean().values
df_full[["전국표준편차_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.groupby(["year", "month"]).std().values
df_full[["서울_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.loc[tmp_df["지점명"] == "서울"].groupby(["year", "month"]).mean().values
df_full[["부산_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.loc[tmp_df["지점명"] == "부산"].groupby(["year", "month"]).mean().values

In [ ]:
df_full.head()

**습도**

In [ ]:
tmp_df = []

for i in glob("C:\\Users\\kogas\\Desktop\\external_data\\20221008\\습도\\*"):
    if i.split("\\")[-1][:2] in ["서울", "인천", "대전", "대구", "광주", "울산", "부산"]:
        tmp = pd.read_csv(i, encoding="cp949")
        tmp = tmp.iloc[:, 1:]
        tmp.columns = ["지점명", "일시", "평균습도", "최저습도"]
        tmp = tmp[["지점명", "일시", "평균습도", "최저습도"]]
        tmp["일시"] = pd.to_datetime(tmp["일시"])
        tmp["year"] = tmp["일시"].dt.year
        tmp["month"] = tmp["일시"].dt.month
        tmp = tmp.loc[(tmp["일시"] >= pd.to_datetime("1996-01-01")) & (tmp["일시"] <= pd.to_datetime("2020-12-31"))]
        tmp_df.append(tmp)
    else:
        continue
#     break

tmp_df = pd.concat(tmp_df, axis=0, ignore_index=True)

In [ ]:
tmp

In [ ]:
df_full[["전국평균_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.groupby(["year", "month"]).mean().values
df_full[["전국표준편차_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.groupby(["year", "month"]).std().values
df_full[["서울_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.loc[tmp_df["지점명"] == "서울"].groupby(["year", "month"]).mean().values
df_full[["부산_" + str(i) for i in tmp_df.groupby(["year", "month"]).mean().columns]] = tmp_df.loc[tmp_df["지점명"] == "부산"].groupby(["year", "month"]).mean().values

In [ ]:
df_full.head()

**소비매출 데이터**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\kosis_korea_retail_sale.csv", encoding="cp949")
tmp_df.isna().sum().sum()

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.head()

In [ ]:
tmp_df["날짜"] = tmp_df["날짜"].apply(lambda x: datetime.datetime.strptime(x, "%Y%m월"))
tmp_df["year"] = tmp_df["날짜"].dt.year
tmp_df["month"] = tmp_df["날짜"].dt.month
df_full[["총합", "내구재", "준내구재", "비내구재"]] = tmp_df[["총합", "내구재", "준내구재", "비내구재"]].values

In [ ]:
df_full.head()

**수출입 데이터**

In [ ]:
# 수출입 데이터
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\kosis_korea_trade_balance.csv")
tmp_df.columns = ["날짜", "수출", "수출_yoy", "수입", "수입_yoy", "trade_balance"]
tmp_df.isna().sum().sum()

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.head()

In [ ]:
tmp_df["날짜"] = tmp_df["날짜"].apply(lambda x: datetime.datetime.strptime(x, "%Y%m월"))
tmp_df["year"] = tmp_df["날짜"].dt.year
tmp_df["month"] = tmp_df["날짜"].dt.month
df_full[["수출", "수출_yoy", "수입_yoy", "trade_balance"]] = tmp_df[["수출", "수출_yoy", "수입_yoy", "trade_balance"]].iloc[1:].values

In [ ]:
df_full.head()

**GDP 데이터**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\kosis_korea_gdp.csv")
tmp_df.columns = ["날짜", "gdp_nominal", "gdp_real_gwr"]
tmp_df.isna().sum().sum()

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.head()

In [ ]:
tmp_df["quarter"] = tmp_df["날짜"].apply(lambda x: "Q" + x[-3])
tmp_df["날짜"] = tmp_df["날짜"].apply(lambda x: datetime.datetime.strptime(x[:4], "%Y"))
tmp_df["year"] = tmp_df["날짜"].dt.year
# tmp_df["month"] = tmp_df["quarter"].apply(lambda x: quarter_dic[x])
tmp_df["month"] = [3, 6, 9, 12] * (104 // 4)

In [ ]:
merge_df = pd.DataFrame(pd.date_range("1996-01", "2020-12", freq="MS"), columns=["날짜"])
merge_df["year"] = merge_df["날짜"].dt.year
merge_df["month"] = merge_df["날짜"].dt.month

In [ ]:
merge_df

In [ ]:
merge_df = merge_df.merge(tmp_df[["year", "month", "gdp_nominal", "gdp_real_gwr"]], on=["year", "month"], how="left")
merge_df["gdp_nominal"] = merge_df["gdp_nominal"].apply(lambda x: str(x).replace(",", "")).astype("float32")
merge_df["gdp_nominal"] = merge_df["gdp_nominal"] / 3
merge_df["gdp_real_gwr"] = (((1 + merge_df["gdp_real_gwr"] / 100) ** (1/3)) - 1) * 100
df_full[["gdp_nominal", "gdp_real_gwr"]] = merge_df[["gdp_nominal", "gdp_real_gwr"]].bfill().ffill().values
del merge_df

In [ ]:
df_full

**Global Energy Price**

In [ ]:
df_full[["GEP"]] = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\Weather_And_GEP.csv")[["GEP"]]

In [ ]:
df_full.head()

**글로벌 수심별 수온편차**

In [ ]:
with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\global_수심별수온편차.txt", "r", encoding="cp949") as f:
    rawdata = f.readlines()
    tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
tmp_df

In [ ]:
tmp_df["연도"] = tmp_df["연도"].astype("int")
tmp_df = tmp_df.loc[tmp_df["연도"]>=1996]
tmp_df["month"] = 12
tmp_df = tmp_df.rename(
    {"연도": "year",
     "국외 수온(수온밑) 편차(0-100m)": "글로벌_수심수온편차_0to100",
     "국외 수온(수온밑) 편차(0-700m)": "글로벌_수심수온편차_0to700",
     "국외 수온(수온밑) 편차(0-2000m)": "글로벌_수심수온편차_0to2000",}, axis=1
)
df_full = df_full.merge(tmp_df[["year", "month", "글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000"]], on=["year", "month"], how="left")
df_full[["글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000"]] = df_full[["글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000"]].astype("float32")
df_full["글로벌_수심수온편차_평균"] = df_full[["글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000"]].mean(axis=1).values
df_full["글로벌_수심수온편차_표준편차"] = df_full[["글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000"]].std(axis=1).values
df_full[["글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000", "글로벌_수심수온편차_평균", "글로벌_수심수온편차_표준편차"]] = df_full[["글로벌_수심수온편차_0to100", "글로벌_수심수온편차_0to700", "글로벌_수심수온편차_0to2000", "글로벌_수심수온편차_평균", "글로벌_수심수온편차_표준편차"]].interpolate().bfill().ffill().values

In [ ]:
df_full.head()

**글로벌 연평균 해양열용량**

In [ ]:
with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\global_연평균_해양열용량.txt", "r", encoding="cp949") as f:
    rawdata = f.readlines()
    tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
tmp_df.replace("", np.nan).dropna()

In [ ]:
tmp_df["연도"] = tmp_df["연도"].astype("int")
tmp_df = tmp_df.loc[tmp_df["연도"]>=1996]
tmp_df["month"] = 12
tmp_df = tmp_df.rename(
    {"연도": "year",
     "국외 해양열용량 편차(0-700m)": "글로벌_해양열용량편차_0to700"}, axis=1
)
df_full = df_full.merge(tmp_df[["year", "month", "글로벌_해양열용량편차_0to700"]], on=["year", "month"], how="left")
df_full[["글로벌_해양열용량편차_0to700"]] = df_full[["글로벌_해양열용량편차_0to700"]].astype("float32")

df_full[["글로벌_해양열용량편차_0to700"]] = df_full[["글로벌_해양열용량편차_0to700"]].interpolate().bfill().ffill().values

In [ ]:
df_full

**글로벌 해수면 높이 및 온도편차 (데이터 부재로 merge 불가)**

In [ ]:
# with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\global_해수면높이_온도편차.txt", "r", encoding="cp949") as f:
#     rawdata = f.readlines()
#     tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
# tmp_df.head()

**글로벌 및 한국 해양표층ph, CO2농도 (데이터 부재로 merge 불가)**

In [ ]:
# with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\korea&global_해양표층ph_co2농도_해양분압.txt", "r", encoding="cp949") as f:
#     rawdata = f.readlines()
#     tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
# tmp_df.replace("", np.nan).dropna()

**한국 3면 연평균 표층염분 (각 데이터 내 분산이 너무 작아서 drop)**

In [ ]:
# with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\korea_3면_연평균_표층염분.txt", "r", encoding="cp949") as f:
#     rawdata = f.readlines()
#     tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
# tmp_df.replace("", np.nan).dropna()

**한국 바다3면 평균수온**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\korea_3면온도.txt", sep=",", encoding="cp949")
# tmp_df.columns = ["날짜", "gdp_nominal", "gdp_real_gwr"]
tmp_df.isna().sum().sum()

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.head()

In [ ]:
tmp_df["연도"] = tmp_df["연도"].astype("int")
tmp_df = tmp_df.loc[tmp_df["연도"]>=1996].reset_index(drop=True)
tmp_df["month"] = 12
tmp_df = tmp_df.rename({"연도": "year", "동해": "해수면온도_동해", "남해": "해수면온도_남해", "서해": "해수면온도_서해"}, axis=1)
df_full = df_full.merge(tmp_df[["year", "month", "해수면온도_동해", "해수면온도_남해", "해수면온도_서해"]], on=["year", "month"], how="left")
df_full["해수면온도_3면평균"] = df_full[["해수면온도_동해", "해수면온도_남해", "해수면온도_서해"]].mean(axis=1)
df_full["해수면온도_3면표준편차"] = df_full[["해수면온도_동해", "해수면온도_남해", "해수면온도_서해"]].std(axis=1)
df_full[["해수면온도_동해", "해수면온도_남해", "해수면온도_서해", "해수면온도_3면평균", "해수면온도_3면표준편차"]] = df_full[["해수면온도_동해", "해수면온도_남해", "해수면온도_서해", "해수면온도_3면평균", "해수면온도_3면표준편차"]].interpolate().bfill().ffill().values

In [ ]:
df_full.head()

**한국 연평균 표층염분 (각 필드의 데이터가 분산이 너무 작아서 drop)**

In [ ]:
# with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\korea_연평균_표층염분.txt", "r", encoding="cp949") as f:
#     rawdata = f.readlines()
#     tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
# tmp_df.replace("", np.nan).dropna()

**한국 연평균 지표온도**

In [ ]:
with open("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\korea_지표온도.txt", "r", encoding="cp949") as f:
    rawdata = f.readlines()
    tmp_df = pd.DataFrame([i.split(",")[:-1] for i in rawdata[1:]], columns=rawdata[0].split(",")[:-1])

In [ ]:
tmp_df.head()

In [ ]:
tmp_df["연도"] = tmp_df["연도"].astype("int")
tmp_df = tmp_df.loc[tmp_df["연도"]>=1996].reset_index(drop=True)
tmp_df["month"] = 12
tmp_df = tmp_df.rename({"연도": "year"}, axis=1)
df_full = df_full.merge(tmp_df[["year", "month", "SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]], on=["year", "month"], how="left")
df_full[["SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]] = df_full[["SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]].astype("float32")
df_full["SF_TMP_고도평균"] = df_full[["SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]].mean(axis=1)
df_full["SF_TMP_고도표준편차"] = df_full[["SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]].std(axis=1)
df_full[["SF_TMP_고도평균", "SF_TMP_고도표준편차", "SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]] = df_full[["SF_TMP_고도평균", "SF_TMP_고도표준편차", "SF_TMP_SUF", "SF_TMP_1m", "SF_TMP_5m"]].interpolate().bfill().ffill().values

In [ ]:
df_full.head()

**한국 해수면높이 및 온도편차**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221006\\korea_해수면높이_온도편차.txt", sep=",", encoding="cp949")
tmp_df.isna().sum().sum()

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.head()

In [ ]:
tmp_df = tmp_df.loc[tmp_df["연도"]>=1996].reset_index(drop=True)
tmp_df["month"] = 12
tmp_df = tmp_df.rename({"연도": "year"}, axis=1)
df_full = df_full.merge(tmp_df[["year", "month", "국내 해수면 높이", "국내 해수면온도 편차"]], on=["year", "month"], how="left")
df_full[["국내 해수면 높이", "국내 해수면온도 편차"]] = df_full[["국내 해수면 높이", "국내 해수면온도 편차"]].interpolate().bfill().ffill().values

In [ ]:
df_full.head()

**유라시아 눈덮힘**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221011\\eurasia_snow_cover.csv")
tmp_df.isna().sum().sum()

In [ ]:
df_full["eurasia_snow_cover"] = tmp_df.loc[(tmp_df["year"] >= 1996) & (tmp_df["year"] <= 2020), "eurasia_snow_cover"].values

In [ ]:
df_full["eurasia_snow_cover"].tail(20)

In [ ]:
# df_full = df_full.drop("eurasia_snow_cover", axis=1)

**음의 북극진동 (2020년도에 na값이 있어 활용이 불가능하다고 판단하여 drop)**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221011\\arctic_oscillation.csv")
tmp_df.isna().sum().sum()

In [ ]:
df_full["arctic_oscillation"] = tmp_df.loc[(tmp_df["Year"] >= 1996) & (tmp_df["Year"] <= 2020), tmp_df.columns[1:]].to_numpy(dtype="float32").flatten()

In [ ]:
df_full["arctic_oscillation"]

In [ ]:
df_full = df_full.drop("arctic_oscillation", axis=1)

**해빙**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221011\\sea_ice_index_monthly_data_by_year_nh_extend.csv")
tmp_df.isna().sum().sum()

In [ ]:
df_full["sea_ice_index"] = tmp_df.loc[(tmp_df["Date"] >= 1996) & (tmp_df["Date"] <= 2020), tmp_df.columns[1:]].to_numpy(dtype="float32").flatten()

In [ ]:
df_full["sea_ice_index"].isna().sum()

**인구 및 인구증가율**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221011\\korea_pop_popGR.csv")
tmp_df.columns = ["year", "korea_pop", "korea_pop_gr"]
tmp_df["month"] = 12

In [ ]:
tmp_df["korea_pop"] = tmp_df["korea_pop"].apply(lambda x: float(x.replace(",", "")))
tmp_df["korea_pop_gr"] = tmp_df["korea_pop_gr"].apply(lambda x: np.nan if x == "-" else float(x.replace(",", "")))

# 성장률을 월간으로 평균한 값을 미리 계산
tmp_df["korea_pop_gr"] = (((1 + tmp_df["korea_pop_gr"] / 100) ** (1/12)) - 1) * 100

In [ ]:
tmp_df.head()

In [ ]:
df_full = df_full.merge(tmp_df, on=["year", "month"], how="left")
df_full["korea_pop"] = df_full["korea_pop"].interpolate().bfill().ffill()
df_full["korea_pop_gr"] = df_full["korea_pop_gr"].bfill().ffill()

In [ ]:
df_full.head()

In [ ]:
df_full.isna().sum().sum()

**글로벌 지역별 천연가스 생산량**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221013\\IEA-world-natural-gas-production-by-region-1973-2020.csv")
# tmp_df.columns = ["year", "korea_pop", "korea_pop_gr"]
tmp_df["month"] = 12

In [ ]:
tmp_df.info()

In [ ]:
tmp_df.head()

In [ ]:
tmp_df[diff(tmp_df.columns, ["year", "month"])] /= 12
df_full = df_full.merge(tmp_df, on=["year", "month"], how="left")
df_full[diff(tmp_df.columns, ["year", "month"])] = df_full[diff(tmp_df.columns, ["year", "month"])].bfill().ffill()

In [ ]:
df_full.head(20)

**KOSIS 데이터 (2000년 부터 데이터가 없음)**

In [ ]:
folder_path_fixed = 'C:\\Users\\kogas\\Desktop\\external_data\\20221105\\external_data_KOSIS\\result_fixed'

In [ ]:
kosis_df = []
for i in glob(folder_path_fixed + "\\*"):
    tmp_df = pd.read_csv(i)
    if (str(tmp_df.iloc[0, 0])[:4] == "2000") & (str(tmp_df.iloc[0, 0])[4:] == "01"): 
        kosis_df.append(tmp_df)

In [ ]:
print(len(kosis_df))

**전력 데이터 (데이터가 듬성듬성 비어있어서 사용 어려움 판단)**

In [ ]:
tmp_df = pd.read_csv("C:\\Users\\kogas\\Desktop\\external_data\\20221105\\kogas_ind_higCorrr.csv", encoding="cp949")

In [ ]:
# tmp_df.dropna()[:30]

In [ ]:
target_vars = ["target_civil", "target_ind"]
target_name = "target_ind"
nontrain_vars = ["year"]
bin_vars = []
cat_vars = ["quarter", "month"]
num_vars = diff(df_full.columns, nontrain_vars + bin_vars + cat_vars)

assert len(df_full.columns) == len(nontrain_vars) + len(cat_vars) + len(num_vars)

## Feature Engineering

- version 0 : qva normalizing feature추가 (기준년도=2015)
- version 1 : short, intm, long에 대해 mean to std 수치 절대값 0.2 이상 feature 선정 (유사 feature는 주관적 판단 하에 drop)
- version 2 : version1 + 시간 feature engineering

In [ ]:
target_vars = ["target_civil", "target_ind"]

**qva normalizing by 2015**

In [ ]:
# 부가가치값 normalizing
# 부가가치 측정의 경우 인플레이션이 반영되어 실제 부가가치가 늘어났는지 정확히 알 수가 없음
# 해당 분기 천연가스 median 가격으로 나눈 후 기준년도(=2015) median 가격을 곱해줌, 2015년도를 기준가격으로 했을 때 얼마나 물건이 생산되었는지 보는 지표
tmp_list = []
for i in ((df_full.groupby(["year", "quarter"]).sum()["qva"] / df_full.groupby(["year", "quarter"]).mean()["GEP"]) * df_full.groupby(["year", "quarter"]).mean().loc[2015, "GEP"]).values:
    tmp_list.extend([i / 3] * 3)
df_full["qva_norm2015"] = tmp_list

In [ ]:
df_full.head()

**time feature engineering (norm, sin, cos)**

In [ ]:
day_secs = 24 * 60 * 60 # 시 분 초
month_secs = (365.2425 / 12) * day_secs
year_secs = (365.2425) * day_secs

In [ ]:
timestamp_s = pd.date_range(str(df_full["year"].unique()[0]) + "-01-01", str(df_full["year"].unique()[-1]) + "-12-31", freq="M").map(datetime.datetime.timestamp)

In [ ]:
df_full["year_sin"] = np.sin(timestamp_s * (2*np.pi / year_secs))
df_full["year_cos"] = np.cos(timestamp_s * (2*np.pi / year_secs))

In [ ]:
df_full["season"] = df_full["month"] % 12 // 3
df_full["year_linear"] = df_full["month"] / 12

cat_vars.append("season")

In [ ]:
plt.plot(df_full["year_sin"].iloc[:36])
plt.plot(df_full["year_cos"].iloc[:36])

In [ ]:
df_full.head()

## 2. EDA

In [ ]:
df_eda = df_full.copy()
df_eda = df_eda.set_index(["year", "quarter", "month"])

In [ ]:
dur_short = 12 * 2
dur_intm = 12 * 10
dur_long = 12 * 14

eda_df_dic = {
    "t0": pd.concat([df_eda, df_eda["target_civil"].shift(-1).to_frame(name="target_civil_t0"), df_eda["target_ind"].shift(-1).to_frame("target_ind_t0")], axis=1).dropna(),
    "short": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_short).to_frame(name="target_civil_short"), df_eda["target_ind"].shift(-dur_short).to_frame("target_ind_short")], axis=1).dropna(),
    "intm": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_intm).to_frame(name="target_civil_intm"), df_eda["target_ind"].shift(-dur_intm).to_frame("target_ind_intm")], axis=1).dropna(),
    "long": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_long).to_frame(name="target_civil_long"), df_eda["target_ind"].shift(-dur_long).to_frame("target_ind_long")], axis=1).dropna(),
}

In [ ]:
eda_df_dic["short"].head()

In [ ]:
eda_df_dic["intm"].head()

In [ ]:
eda_df_dic["long"].head()

### 산업용

**correlation table**

In [ ]:
df_corr = pd.concat([
    eda_df_dic["t0"].corr().iloc[:-2, -2:],
    eda_df_dic["short"].corr().iloc[:-2, -2:],
    eda_df_dic["intm"].corr().iloc[:-2, -2:],
    eda_df_dic["long"].corr().iloc[:-2, -2:],
], axis=1)
df_corr = {
    "target_civil": df_corr.filter(regex="target_civil"),
    "target_ind":  df_corr.filter(regex="target_ind"),
}

In [ ]:
# dur_vars = ["target_civil_t0", "target_civil_short", "target_civil_intm", "target_civil_long"]
# df_corr["target_civil"]["target_civil_mean"] = df_corr["target_civil"][dur_vars].mean(axis=1)
# df_corr["target_civil"]["target_civil_std"] = df_corr["target_civil"][dur_vars].std(axis=1)
# df_corr["target_civil"]["target_civil_meantostd"] = df_corr["target_civil"]["target_civil_mean"] / (df_corr["target_civil"]["target_civil_std"] + 1)
# df_corr["target_civil"]["target_civil_longtoshort"] = df_corr["target_civil"]["target_civil_long"].abs() / (df_corr["target_civil"]["target_civil_short"].abs() + 1)
# df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_short"].abs().values)[::-1]]
# # df_summary.loc[df_summary["target_civil_meantostd"].abs() > 0.2]
# df_summary.iloc[:20]

In [ ]:
# dur_vars = ["target_ind_t0", "target_ind_short", "target_ind_intm", "target_ind_long"]
dur_vars = ["target_ind_t0", "target_ind_short", "target_ind_intm", "target_ind_long"]
df_corr["target_ind"]["target_ind_mean"] = df_corr["target_ind"][dur_vars].abs().mean(axis=1)
df_corr["target_ind"]["target_ind_std"] = df_corr["target_ind"][dur_vars].std(axis=1)
df_corr["target_ind"]["target_ind_meantostd"] = df_corr["target_ind"]["target_ind_mean"] / (df_corr["target_ind"]["target_ind_std"] + 1)
df_corr["target_ind"]["target_ind_longtoshort"] = df_corr["target_ind"]["target_ind_long"].abs() / (df_corr["target_ind"]["target_ind_short"].abs() + 1)

df_summary = df_corr["target_ind"].round(4).iloc[np.argsort(df_corr["target_ind"]["target_ind_meantostd"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_ind_mean", "target_ind_std", "target_ind_meantostd", "target_ind_longtoshort"]].iloc[:50])
# base feature : mean to std 0.35 이상만

In [ ]:
df_summary = df_corr["target_ind"].round(4).iloc[np.argsort(df_corr["target_ind"]["target_ind_t0"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_ind_mean", "target_ind_std", "target_ind_meantostd", "target_ind_longtoshort"]].iloc[:30])
t0_term_vars = ["target_ind", "수출", "qva", "Non-OECD Asia (Incl. China)", "korea_pop"]

In [ ]:
df_summary = df_corr["target_ind"].round(4).iloc[np.argsort(df_corr["target_ind"]["target_ind_short"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_ind_mean", "target_ind_std", "target_ind_meantostd", "target_ind_longtoshort"]].iloc[:30])
short_term_vars = ["target_ind", "Non-OECD Asia (Incl. China)", "qva", "korea_pop", "수출"]

In [ ]:
df_summary = df_corr["target_ind"].round(4).iloc[np.argsort(df_corr["target_ind"]["target_ind_intm"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_ind_mean", "target_ind_std", "target_ind_meantostd", "target_ind_longtoshort"]].iloc[:30])
intm_term_vars = ["target_civil", "target_ind", "korea_pop", "OECD", "SF_TMP_5m"]

In [ ]:
df_summary = df_corr["target_ind"].round(4).iloc[np.argsort(df_corr["target_ind"]["target_ind_long"].abs().values)[::-1]]
# df_summary.loc[df_summary["target_ind_meantostd"].abs() > 0.2]
# df_summary.iloc[:20]
display(df_summary[dur_vars + ["target_ind_mean", "target_ind_std", "target_ind_meantostd", "target_ind_longtoshort"]].iloc[:30])
long_term_vars = ["year_cos", "서울_최고기온", "eurasia_snow_cover", "부산_평균습도", "target_civil"]

### EDA for short term, long term threshold

In [ ]:
# dur_short = 12 * 2
# dur_intm = 12 * 8
# dur_long = 12 * 14

# eda_df_dic = {
#     "t0": pd.concat([df_eda, df_eda["target_civil"].shift(-1).to_frame(name="target_civil_t0"), df_eda["target_ind"].shift(-1).to_frame("target_ind_t0")], axis=1).dropna(),
#     "short": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_short).to_frame(name="target_civil_short"), df_eda["target_ind"].shift(-dur_short).to_frame("target_ind_short")], axis=1).dropna(),
#     "intm": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_intm).to_frame(name="target_civil_intm"), df_eda["target_ind"].shift(-dur_intm).to_frame("target_ind_intm")], axis=1).dropna(),
#     "long": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_long).to_frame(name="target_civil_long"), df_eda["target_ind"].shift(-dur_long).to_frame("target_ind_long")], axis=1).dropna(),
# }

In [ ]:
for i in range(1, 169, 1):
    eda_df_dic["t" + str(i)] = pd.concat([
        df_eda,
        df_eda["target_civil"].shift(-i).to_frame(name="target_civil_t" + str(i)),
        df_eda["target_ind"].shift(-i).to_frame("target_ind_t" + str(i))
    ], axis=1).dropna()

In [ ]:
tmp_shorterm = []
tmp_longterm = []

for i in range(1, 169, 1):
    df_corr = eda_df_dic["t" + str(i)].copy()
    tmp_shorterm.append(df_corr.corr().iloc[:-2, -2:].loc[short_term_vars, "target_ind_t" + str(i)].abs().mean())
    tmp_longterm.append(df_corr.corr().iloc[:-2, -2:].loc[long_term_vars, "target_ind_t" + str(i)].abs().mean())
#     df_corr["target_civil"]["target_civil_mean"] = df_corr["target_civil"][dur_vars].mean(axis=1)
#     df_corr["target_civil"]["target_civil_std"] = df_corr["target_civil"][dur_vars].std(axis=1)
#     df_corr["target_civil"]["target_civil_meantostd"] = df_corr["target_civil"]["target_civil_mean"] / (df_corr["target_civil"]["target_civil_std"] + 1)
#     df_corr["target_civil"]["target_civil_longtoshort"] = df_corr["target_civil"]["target_civil_long"].abs() / (df_corr["target_civil"]["target_civil_short"].abs() + 1)
#     break
    

In [ ]:
plt.figure(figsize=(16, 8))

plt.plot(
    ["YEAR+" + str(i+1) for i in range(14)],
    [np.mean(tmp_shorterm[((i+1)*12-12):((i+1)*12)]) for i in range(14)],
    color="orange", marker="o", label="short term"
)
plt.plot(
    ["YEAR+" + str(i+1) for i in range(14)],
    [np.mean(tmp_longterm[((i+1)*12-12):((i+1)*12)]) for i in range(14)],
    linestyle="--", color="green", marker="o", label="long term"
)

plt.legend(fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(tmp_shorterm)
plt.show()

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(tmp_longterm)
plt.show()

### 민간용

**correlation table**

In [ ]:
df_corr = pd.concat([
    eda_df_dic["t0"].corr().iloc[:-2, -2:],
    eda_df_dic["short"].corr().iloc[:-2, -2:],
    eda_df_dic["intm"].corr().iloc[:-2, -2:],
    eda_df_dic["long"].corr().iloc[:-2, -2:],
], axis=1)
df_corr = {
    "target_civil": df_corr.filter(regex="target_civil"),
    "target_ind":  df_corr.filter(regex="target_ind"),
}

In [ ]:
# dur_vars = ["target_civil_t0", "target_civil_short", "target_civil_intm", "target_civil_long"]
# df_corr["target_civil"]["target_civil_mean"] = df_corr["target_civil"][dur_vars].mean(axis=1)
# df_corr["target_civil"]["target_civil_std"] = df_corr["target_civil"][dur_vars].std(axis=1)
# df_corr["target_civil"]["target_civil_meantostd"] = df_corr["target_civil"]["target_civil_mean"] / (df_corr["target_civil"]["target_civil_std"] + 1)
# df_corr["target_civil"]["target_civil_longtoshort"] = df_corr["target_civil"]["target_civil_long"].abs() / (df_corr["target_civil"]["target_civil_short"].abs() + 1)
# df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_short"].abs().values)[::-1]]
# # df_summary.loc[df_summary["target_civil_meantostd"].abs() > 0.2]
# df_summary.iloc[:20]

In [ ]:
df_corr["target_civil"]

In [ ]:
# dur_vars = ["target_civil_t0", "target_civil_short", "target_civil_intm", "target_civil_long"]
dur_vars = ["target_civil_t0", "target_civil_short", "target_civil_intm", "target_civil_long"]
df_corr["target_civil"]["target_civil_mean"] = df_corr["target_civil"][dur_vars].abs().mean(axis=1)
df_corr["target_civil"]["target_civil_std"] = df_corr["target_civil"][dur_vars].std(axis=1)
df_corr["target_civil"]["target_civil_meantostd"] = df_corr["target_civil"]["target_civil_mean"] / (df_corr["target_civil"]["target_civil_std"] + 1)
df_corr["target_civil"]["target_civil_longtoshort"] = df_corr["target_civil"]["target_civil_long"].abs() / (df_corr["target_civil"]["target_civil_short"].abs() + 1)

df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_meantostd"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_civil_mean", "target_civil_std", "target_civil_meantostd", "target_civil_longtoshort"]].iloc[:50])
# base feature : mean to std 0.35 이상만

In [ ]:
df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_t0"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_civil_mean", "target_civil_std", "target_civil_meantostd", "target_civil_longtoshort"]].iloc[:30])
t0_term_vars = ["year_cos", "target_civil", "서울_최고기온", "eurasia_snow_cover", "부산_평균습도"]

In [ ]:
df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_short"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_civil_mean", "target_civil_std", "target_civil_meantostd", "target_civil_longtoshort"]].iloc[:30])
short_term_vars = ["target_civil", "전국평균_최고기온", "eurasia_snow_cover", "부산_평균습도", "year_cos"]

In [ ]:
df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_intm"].abs().values)[::-1]]
display(df_summary[dur_vars + ["target_civil_mean", "target_civil_std", "target_civil_meantostd", "target_civil_longtoshort"]].iloc[:30])
intm_term_vars = ["전국평균_최고기온", "eurasia_snow_cover", "target_civil", "부산_평균습도", "year_cos"]

In [ ]:
df_summary = df_corr["target_civil"].round(4).iloc[np.argsort(df_corr["target_civil"]["target_civil_long"].abs().values)[::-1]]
# df_summary.loc[df_summary["target_civil_meantostd"].abs() > 0.2]
# df_summary.iloc[:20]
display(df_summary[dur_vars + ["target_civil_mean", "target_civil_std", "target_civil_meantostd", "target_civil_longtoshort"]].iloc[:30])
long_term_vars = ["전국평균_최고기온", "eurasia_snow_cover", "부산_평균습도", "target_civil", "weight_civil"]

### EDA for short term, long term threshold

In [ ]:
# dur_short = 12 * 2
# dur_intm = 12 * 8
# dur_long = 12 * 14

# eda_df_dic = {
#     "t0": pd.concat([df_eda, df_eda["target_civil"].shift(-1).to_frame(name="target_civil_t0"), df_eda["target_ind"].shift(-1).to_frame("target_ind_t0")], axis=1).dropna(),
#     "short": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_short).to_frame(name="target_civil_short"), df_eda["target_ind"].shift(-dur_short).to_frame("target_ind_short")], axis=1).dropna(),
#     "intm": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_intm).to_frame(name="target_civil_intm"), df_eda["target_ind"].shift(-dur_intm).to_frame("target_ind_intm")], axis=1).dropna(),
#     "long": pd.concat([df_eda, df_eda["target_civil"].shift(-dur_long).to_frame(name="target_civil_long"), df_eda["target_ind"].shift(-dur_long).to_frame("target_ind_long")], axis=1).dropna(),
# }

In [ ]:
for i in range(1, 169, 1):
    eda_df_dic["t" + str(i)] = pd.concat([
        df_eda,
        df_eda["target_ind"].shift(-i).to_frame(name="target_ind_t" + str(i)),
        df_eda["target_civil"].shift(-i).to_frame(name="target_civil_t" + str(i))
    ], axis=1).dropna()

In [ ]:
tmp_shorterm = []
tmp_longterm = []

for i in range(1, 169, 1):
    df_corr = eda_df_dic["t" + str(i)].copy()
    tmp_shorterm.append(df_corr.corr().iloc[:-2, -2:].loc[short_term_vars, "target_civil_t" + str(i)].abs().mean())
    tmp_longterm.append(df_corr.corr().iloc[:-2, -2:].loc[long_term_vars, "target_civil_t" + str(i)].abs().mean())
#     df_corr["target_civil"]["target_civil_mean"] = df_corr["target_civil"][dur_vars].mean(axis=1)
#     df_corr["target_civil"]["target_civil_std"] = df_corr["target_civil"][dur_vars].std(axis=1)
#     df_corr["target_civil"]["target_civil_meantostd"] = df_corr["target_civil"]["target_civil_mean"] / (df_corr["target_civil"]["target_civil_std"] + 1)
#     df_corr["target_civil"]["target_civil_longtoshort"] = df_corr["target_civil"]["target_civil_long"].abs() / (df_corr["target_civil"]["target_civil_short"].abs() + 1)
#     break
    

In [ ]:
plt.figure(figsize=(16, 8))

plt.plot(
    ["YEAR+" + str(i+1) for i in range(14)],
    [np.mean(tmp_shorterm[((i+1)*12-12):((i+1)*12)]) for i in range(14)],
    color="orange", marker="o", label="short term"
)
plt.plot(
    ["YEAR+" + str(i+1) for i in range(14)],
    [np.mean(tmp_longterm[((i+1)*12-12):((i+1)*12)]) for i in range(14)],
    linestyle="--", color="green", marker="o", label="long term"
)

plt.legend(fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(tmp_shorterm)
plt.show()

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(tmp_longterm)
plt.show()

**2-1. Distributin on Target (민간용)**

* 민간용 수요는 2005년까지 점진적으로 증가세를 보인다. 그 이후에는 횡보하는 모습이다.

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot([str(i[0]) + "-" + str(i[1]) for i in df_eda.groupby(["year", "quarter"]).mean().index.values], df_eda.groupby(["year", "quarter"]).mean()["target_civil"].values, marker="o", mfc="orange")
plt.title("Distribution on civil's natural gas demand", pad=20, fontsize=14, fontweight="bold")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot([str(i) for i in df_eda.groupby(["year"]).mean().index.values], df_eda.groupby(["year"]).mean()["target_civil"].values, marker="o", mfc="orange")
plt.title("Distribution on civil's natural gas demand", pad=20, fontsize=14, fontweight="bold")
plt.xticks(rotation=90)
plt.show()

**2-2. Distributin on Target (산업용)**

* 산업용 수요는 2012년까지 급격한 상승세를 보이다가 이후 살짝 감소 및 횡보하는 모습이다.

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot([str(i[0]) + "-" + str(i[1]) for i in df_eda.groupby(["year", "quarter"]).mean().index.values], df_eda.groupby(["year", "quarter"]).mean()["target_ind"].values, marker="o", mfc="orange")
plt.title("Distribution on industry's natural gas demand", pad=20)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot([str(i) for i in df_eda.groupby(["year"]).mean().index.values], df_eda.groupby(["year"]).mean()["target_ind"].values, marker="o", mfc="orange")
plt.title("Distribution on civil's natural gas demand", pad=20, fontsize=14, fontweight="bold")
plt.xticks(rotation=90)
plt.show()

**2-3. Distribution on features by timeseries**

* 산업용대비 민간용 수요는 점차 줄고 있다. (민간용은 그래도이나 산업용 수요가 2012년까지 점차 증가했기 때문)
* 산업용 부가가치는 지속적으로 증가 중 (명목가치로 계산된 것으로 보여 실질적으로 증가했는지는 추가 작업을 통해 확인해야함)
* 천연가스 가격은 2013년 까지 상승 후 하락
* 오일 가격은 2011년 까지 상승 후 하락

In [ ]:
for i in df_eda.columns:
    if i in target_vars:
        continue
    plt.figure(figsize=(20, 9))
    plt.plot([str(i[0]) + "-" + str(i[1]) for i in df_eda.groupby(["year", "quarter"]).mean().index.values], df_eda.groupby(["year", "quarter"]).mean()[i].values, marker="o", mfc="orange")
    plt.title("Distribution on " + i, pad=20)
    plt.xticks(rotation=90)
    plt.show()

**2-4. Scatter plot on feature & target (민간용) - short**
* 상대가격은 민간용 수요와 크게 상관성이 없음
* 산업용 수요와 비교했을 시, 민간용 수요는 가격과 큰 상관성이 없다고 보여짐
* **기후관련 feature는 큰 상관성이 있을 것이라고 기대됨**
* 바로 다음 월과의 상관관계보다 같은 월에 대해 더 큰 상관관계를 가짐 (계절 영향일 것으로 보임)

In [ ]:
for i in eda_df_dic["short"].columns:
    if i in ["target_civil_short", "target_ind_short"]:
        continue
    plt.figure(figsize=(12, 8))
    plt.scatter(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_civil_short"].values, color="orange", edgecolor="black")
    corr = np.corrcoef(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_civil_short"].values)[0,1]
    plt.title(f"Scatter plot on {i} (민간용 short term) - (Correlation={np.round(corr, 3)})", pad=20, weight="bold", fontsize=14)
    x_range = plt.xlim()
    y_range = plt.ylim()
    reg = linregress(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_civil_short"].values)
    corr = np.corrcoef(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_civil_short"].values)[0,1]
    plt.axline(xy1=(0, reg.intercept), slope=reg.slope, linestyle="--", color="green")
    plt.xlim(x_range)
#     plt.text(x_range[0] + 0.1 * np.abs(x_range[0]), y_range[1] - 0.1 * np.abs(y_range[1]), s=f"Correlation : {np.round(corr, 3)}", weight="bold", fontsize=12)
    plt.show()

**2-4. Scatter plot on feature & target (민간용) - long**

In [ ]:
for i in eda_df_dic["long"].columns:
    if i in ["target_civil_long", "target_ind_long"]:
        continue
    plt.figure(figsize=(12, 8))
    plt.scatter(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_civil_long"].values, color="orange", edgecolor="black")
    corr = np.corrcoef(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_civil_long"].values)[0,1]
    plt.title(f"Scatter plot on {i} (민간용 long term) - (Correlation : {np.round(corr, 3)})", pad=20, weight="bold", fontsize=14)
    x_range = plt.xlim()
    y_range = plt.ylim()
    reg = linregress(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_civil_long"].values)
    corr = np.corrcoef(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_civil_long"].values)[0,1]
    plt.axline(xy1=(0, reg.intercept), slope=reg.slope, linestyle="--", color="green")
    plt.xlim(x_range)
#     plt.text(x_range[0] + 0.1 * np.abs(x_range[0]), y_range[1] - 0.1 * np.abs(y_range[1]), s=f"Correlation : {np.round(corr, 3)}", weight="bold", fontsize=12)
    plt.show()

**2-5. Scatter plot on feature & target (산업용) - short**

* 산업용부가가치와 큰 양의 상관관계를 가짐 
* 천연가스 가격 뿐만 아닌 오일 가격과도 큰 양의 상관관계를 가짐
* **산업용 부분은 기후보다 경제상황과 관련된 feature와 큰 상관성을 가질 것이라고 기대 됨**

In [ ]:
for i in eda_df_dic["short"].columns:
    if i in ["target_civil_short", "target_ind_short"]:
        continue
    plt.figure(figsize=(12, 8))
    plt.scatter(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_ind_short"].values, color="orange", edgecolor="black")
    corr = np.corrcoef(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_ind_short"].values)[0,1]
    plt.title(f"Scatter plot on {i} (산업용 short term) - (Correlation : {np.round(corr, 3)})", pad=20, weight="bold", fontsize=14)
    x_range = plt.xlim()
    y_range = plt.ylim()
    reg = linregress(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_ind_short"].values)
    corr = np.corrcoef(eda_df_dic["short"][i].values, eda_df_dic["short"]["target_ind_short"].values)[0,1]
    plt.axline(xy1=(0, reg.intercept), slope=reg.slope, linestyle="--", color="green")
    plt.xlim(x_range)
#     plt.text(x_range[0] + 0.1 * np.abs(x_range[0]), y_range[1] - 0.1 * np.abs(y_range[1]), s=f"Correlation : {np.round(corr, 3)}", weight="bold", fontsize=12)
    plt.show()

**2-5. Scatter plot on feature & target (산업용) - long**

In [ ]:
for i in eda_df_dic["long"].columns:
    if i in ["target_civil_long", "target_ind_long"]:
        continue
    plt.figure(figsize=(12, 8))
    plt.scatter(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_ind_long"].values, color="orange", edgecolor="black")
    corr = np.corrcoef(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_ind_long"].values)[0,1]
    plt.title(f"Scatter plot on {i} (산업용 short term) - (Correlation : {np.round(corr, 3)})", pad=20, weight="bold", fontsize=14)
    x_range = plt.xlim()
    y_range = plt.ylim()
    reg = linregress(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_ind_long"].values)
    corr = np.corrcoef(eda_df_dic["long"][i].values, eda_df_dic["long"]["target_ind_long"].values)[0,1]
    plt.axline(xy1=(0, reg.intercept), slope=reg.slope, linestyle="--", color="green")
    plt.xlim(x_range)
#     plt.text(x_range[0] + 0.1 * np.abs(x_range[0]), y_range[1] - 0.1 * np.abs(y_range[1]), s=f"Correlation : {np.round(corr, 3)}", weight="bold", fontsize=12)
    plt.show()

**create data pipeline**

In [ ]:
df_full.isna().sum().sum()

In [ ]:
nontrain_vars = ["year"]
bin_vars = []
cat_vars = ["quarter", "month"]
num_vars = diff(df_full.columns, target_vars + nontrain_vars + bin_vars + cat_vars)

In [ ]:
df_full[num_vars] = df_full[num_vars].astype("float32")

In [ ]:
pred_steps = 168
seqLength = 12 + 1

def get_multilabel(df):
    concat_list = []
    for i in range(len(df)):
        valid_split_point = len(df) - pred_steps - i
        if valid_split_point < 1:
            valid_split_point += 1
            i -= 1
            break
        tmp_target = df[target_vars].iloc[valid_split_point:(len(df)-i)]
        tmp_civil = tmp_target["target_civil"].to_frame().T
        tmp_civil.columns = ["target_civil_t" + str(i) for i in range(pred_steps)]

        tmp_ind = tmp_target["target_ind"].to_frame().T
        tmp_ind.columns = ["target_ind_t" + str(i) for i in range(pred_steps)]

        concat_list.append(
            pd.concat([
                df.iloc[valid_split_point-1].to_frame().T.reset_index(drop=True),
                tmp_civil.reset_index(drop=True), tmp_ind.reset_index(drop=True)
            ], axis=1)
        )
    return pd.concat(concat_list, axis=0, ignore_index=True)[::-1].reset_index(drop=True)

In [ ]:
ref_date_range = pd.Index(df_full[["year", "quarter", "month"]].iloc[(seqLength-1):])
df_full = df_full.drop(nontrain_vars, axis=1)

ohe = OneHotEncoder(sparse=False)
df_full = pd.concat([df_full.drop(cat_vars, axis=1), pd.DataFrame(ohe.fit_transform(df_full[cat_vars]))], axis=1)

In [ ]:
# df_test = df_full.iloc[(-pred_steps-seqLength+1):]
df_test = df_full.iloc[-pred_steps:]
df_full = get_multilabel(df_full)

In [ ]:
df_full

In [ ]:
df_test

## Modeling - baseline

In [ ]:
architecture_name = "fv" + str(feature_version) + "_linear_targetTF_outputMed_try2"
architecture_root_path = folder_path + "architectures\\" + architecture_name + "\\"
if not os.path.exists(architecture_root_path):
    os.makedirs(architecture_root_path)
shutil.copy(folder_path + "YJ_baseline_noSeq_targetTF.ipynb", architecture_root_path + "YJ_baseline_noSeq_targetTF.ipynb")

### 민간용 부분 training & inference

**learning parameter setting**

In [ ]:
def do_fold_training(target_name, another_target_name):
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(df_full)):
        scaler = StandardScaler()
        target_scaler = StandardScaler()

        train_ds = df_full.iloc[train_idx].reset_index(drop=True)
    #     train_ds[num_vars + ["target_ind"]] = scaler.fit_transform(train_ds[num_vars + ["target_ind"]])
    #     train_ds[[target_name]] = target_scaler.fit_transform(train_ds[[target_name]])
        train_ds[scaling_vars] = np.log1p(train_ds[scaling_vars])
        train_ds[[target_name]] = np.log1p(train_ds[[target_name]])


        valid_ds = df_full.iloc[valid_idx]
    #     valid_ds[num_vars + ["target_ind"]] = scaler.transform(valid_ds[num_vars + ["target_ind"]])
    #     valid_ds[[target_name]] = target_scaler.transform(valid_ds[[target_name]])
        valid_ds[scaling_vars] = np.log1p(valid_ds[scaling_vars])
        valid_ds[[target_name]] = np.log1p(valid_ds[[target_name]])

        test_ds = df_test.iloc[:]
    #     test_ds[num_vars + ["target_ind"]] = scaler.transform(test_ds[num_vars + ["target_ind"]])
    #     test_ds[[target_name]] = target_scaler.transform(test_ds[[target_name]])
        test_ds[scaling_vars] = np.log1p(test_ds[scaling_vars])
        test_ds[[target_name]] = np.log1p(test_ds[[target_name]])

        base_model = LinearRegression()
        chain_model = RegressorChain(base_model, cv=None, random_state=42)

        target_df = train_ds.filter(regex=target_name + "_t").to_numpy()
    #     chain_model.fit(
    #         train_ds[diff(train_ds.columns, list(train_ds.filter(regex='target_civil_t').columns) + list(train_ds.filter(regex='target_ind_t').columns))].to_numpy(),
    #         np.concatenate([target_scaler.transform(target_df[:,[i]]) for i in range(target_df.shape[1])], axis=1)
    #     )
        chain_model.fit(
            train_ds[diff(train_ds.columns, list(train_ds.filter(regex='target_civil_t').columns) + list(train_ds.filter(regex='target_ind_t').columns))].to_numpy(),
            np.concatenate([np.log1p(target_df[:,[i]]) for i in range(target_df.shape[1])], axis=1)
        )

        tmp_pred = chain_model.predict(
            valid_ds[diff(valid_ds.columns, list(valid_ds.filter(regex='target_civil_t').columns) + list(valid_ds.filter(regex='target_ind_t').columns))].to_numpy()
        )
    #     valid_pred = np.concatenate([target_scaler.inverse_transform(tmp_pred[:,[i]]) for i in range(tmp_pred.shape[1])], axis=1)
        valid_pred = np.concatenate([np.expm1(tmp_pred[:,[i]]) for i in range(tmp_pred.shape[1])], axis=1)

        tmp_pred = chain_model.predict(
            test_ds[diff(test_ds.columns, list(test_ds.filter(regex='target_civil_t').columns) + list(test_ds.filter(regex='target_ind_t').columns))].to_numpy()
        )
    #     test_pred[target_name][:] += np.concatenate([target_scaler.inverse_transform(tmp_pred[:,[i]]) for i in range(tmp_pred.shape[1])], axis=1) / n_folds
        test_pred[target_name][:] += np.concatenate([np.expm1(tmp_pred[:,[i]]) for i in range(tmp_pred.shape[1])], axis=1) / n_folds

        metric_list["MAE"].append(skl_merics.mean_absolute_error(valid_pred, valid_ds.filter(regex=target_name + "_t").to_numpy()))
        metric_list["MAPE"].append(skl_merics.mean_absolute_percentage_error(valid_pred, valid_ds.filter(regex=target_name + "_t").to_numpy(), multioutput="uniform_average"))

        scaler_list.append(scaler)
        model_list.append(chain_model)

In [ ]:
test_pred = {
    "target_civil": np.zeros(shape=(pred_steps, pred_steps)),
    "target_ind": np.zeros(shape=(pred_steps, pred_steps))
}

In [ ]:
# 맨 마지막 fold가 가장 최신임
n_folds = 4
# group_kfolds_dic = get_cv_dic(n_folds=n_folds, ref_idx=np.array(range(len(df_full_x))))
kfolds_spliter = TimeSeriesSplit(n_folds, max_train_size=60, test_size=3)

target_name = "target_civil"
another_target_name = "target_ind"
scaling_vars = ["qva", "gas_price", "oil_price"] + [target_name, another_target_name]

scaler_list = []
model_list = []
metric_list = {
    "MAE": [],
    "MAPE": []
}

seed_everything()
do_fold_training(target_name, another_target_name)

In [ ]:
score_table = pd.DataFrame(metric_list)
score_table.loc["average"] = score_table.mean(axis=0)
score_table.loc["std"] = score_table.std(axis=0)
score_table.to_csv(architecture_root_path + target_name + "_score_table.csv", index=True)

In [ ]:
score_table

### 산업용 부분 training & inference

In [ ]:
# 맨 마지막 fold가 가장 최신임
n_folds = 4
# group_kfolds_dic = get_cv_dic(n_folds=n_folds, ref_idx=np.array(range(len(df_full_x))))
kfolds_spliter = TimeSeriesSplit(n_folds, max_train_size=60, test_size=3)

target_name = "target_ind"
another_target_name = "target_civil"
scaling_vars = ["qva", "gas_price", "oil_price"] + [target_name, another_target_name]

scaler_list = []
model_list = []
metric_list = {
    "MAE": [],
    "MAPE": []
}

seed_everything()
do_fold_training(target_name, another_target_name)

In [ ]:
score_table = pd.DataFrame(metric_list)
score_table.loc["average"] = score_table.mean(axis=0)
score_table.loc["std"] = score_table.std(axis=0)
score_table.to_csv(architecture_root_path + target_name + "_score_table.csv", index=True)

In [ ]:
score_table

In [ ]:
df_submission = pd.read_csv('C:\\Users\\kogas\\Desktop\\task1_files\\dataset\\submission_sample.csv')
df_submission.info()

## Submission

In [ ]:
df_submission = pd.read_csv('C:\\Users\\kogas\\Desktop\\task1_files\\dataset\\submission_sample.csv')
df_submission.head()

### 민간용 submission

In [ ]:
fliped_output = np.fliplr(test_pred["target_civil"])
output_list = []
for i in range(test_pred["target_civil"].shape[0]):
#     output_list.append(round(trim_mean(fliped_output.diagonal(-i), proportiontocut=0.25), 3))
    output_list.append(round(np.median(fliped_output.diagonal(-i)), 3))
#     output_list.append(round(pd.Series(fliped_output.diagonal(-i)).ewm(alpha=0.5, min_periods=len(fliped_output.diagonal(-i))).mean().iloc[-1], 3))

In [ ]:
df_submission["CIVIL"] = output_list

### 산업용 submission

In [ ]:
fliped_output = np.fliplr(test_pred["target_ind"])
output_list = []
for i in range(test_pred["target_ind"].shape[0]):
#     output_list.append(round(trim_mean(fliped_output.diagonal(-i), proportiontocut=0.25), 3))
    output_list.append(round(np.median(fliped_output.diagonal(-i)) , 3))
#     output_list.append(round(pd.Series(fliped_output.diagonal(-i)).ewm(alpha=0.5, min_periods=len(fliped_output.diagonal(-i))).mean().iloc[-1], 3))

In [ ]:
output_list[:5]

In [ ]:
df_submission["IND"] = output_list

In [ ]:
df_submission

In [ ]:
df_submission.to_csv(architecture_root_path + architecture_name + ".csv", index=False)

## Visualization on inference data

**create dataframe for visualization**

In [ ]:
df_result_viz = df_submission.copy()
df_result_viz.columns = df_result_viz.columns.str.lower()
df_result_viz["quarter"] = df_result_viz["month"].apply(lambda x: month_dic[x])
df_result_viz

**for civil (분기 평균)**

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(
    [str(i[0]) + "-" + str(i[1]) for i in df_eda.loc[df_eda.index.get_level_values(0) >= 2010].groupby(["year", "quarter"]).mean().index.values],
    df_eda.loc[df_eda.index.get_level_values(0) >= 2010].groupby(["year", "quarter"]).mean()["target_civil"].values,
    marker="o", mfc="orange"
)
plt.plot(
    [str(i[0]) + "-" + str(i[1]) for i in df_result_viz.groupby(["year", "quarter"]).mean().index.values],
    df_result_viz.groupby(["year", "quarter"]).mean()["civil"].values,
    color="grey", linestyle="--", marker="o", mfc="green"
)
plt.title("forecasting on civil natural gas demand", pad=20, fontsize=14, weight="bold")
plt.xticks(rotation=90)
plt.show()

**for industrial (분기 평균)**

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(
    [str(i[0]) + "-" + str(i[1]) for i in df_eda.loc[df_eda.index.get_level_values(0) >= 2010].groupby(["year", "quarter"]).mean().index.values],
    df_eda.loc[df_eda.index.get_level_values(0) >= 2010].groupby(["year", "quarter"]).mean()["target_ind"].values,
    marker="o", mfc="orange"
)
plt.plot(
    [str(i[0]) + "-" + str(i[1]) for i in df_result_viz.groupby(["year", "quarter"]).mean().index.values],
    df_result_viz.groupby(["year", "quarter"]).mean()["ind"].values,
    color="grey", linestyle="--", marker="o", mfc="green"
)
plt.title("forecasting on industrial natural gas demand", pad=20, fontsize=14, weight="bold")
plt.xticks(rotation=90)
plt.show()